In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
               'price':'{:.2f}','buy_p':'{:.2f}','sell_p':'{:.2f}','diff':'{:.2f}',
               'ppu':'{:.4f}',
               'net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','profit':'{:,.2f}',
               'div_amt':'{:,.2f}','ttl_amt':'{:,.2f}','pct':'{:,.2f}',
               'qty':'{:,}','ttl_qty':'{:,}','number':'{:,}','days':'{:,}'
              }
pd.set_option("display.max_rows", None)

cols = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter'.split()
colt = 'name ttl_qty buy_amt ppu div_amt buy_date x_date p_date days pct year quarter number'.split()

name = 'DIF'

### 1st Batch

In [2]:
div_year = 2019
div_quarter = 2
ids = '637'

In [3]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)

,name,ttl_qty,buy_amt,buy_date
0,DIF,"7,000.0","103,127.91",2019-07-24


In [4]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,235,DIF,2021,1,"100,000",0.2610,"26,100.00",2021-05-12,2021-06-04
1,197,DIF,2020,4,"80,000",0.2610,"20,880.00",2021-02-23,2021-03-12
2,190,DIF,2020,3,"70,000",0.2610,"18,270.00",2020-11-11,2020-12-04
3,3,DIF,2020,2,"60,000",0.2610,"15,660.00",2020-08-13,2020-09-08
4,6,DIF,2020,1,"52,000",0.2610,"13,572.00",2020-05-26,2020-06-11
5,31,DIF,2019,4,"52,000",0.2610,"13,572.00",2020-02-12,2020-03-03
6,36,DIF,2019,3,"52,000",0.1656,"8,611.20",2019-11-18,2019-12-04
7,43,DIF,2019,2,"47,000",0.3469,"16,304.30",2019-08-26,2019-09-11


In [5]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,DIF,7000.0,103127.91,0.2610,1827.0,2019-07-24,2021-05-12,2021-06-04,681,0.95,2021,1
1,DIF,7000.0,103127.91,0.2610,1827.0,2019-07-24,2021-02-23,2021-03-12,597,1.08,2020,4
2,DIF,7000.0,103127.91,0.2610,1827.0,2019-07-24,2020-11-11,2020-12-04,499,1.30,2020,3
3,DIF,7000.0,103127.91,0.2610,1827.0,2019-07-24,2020-08-13,2020-09-08,412,1.57,2020,2
4,DIF,7000.0,103127.91,0.2610,1827.0,2019-07-24,2020-05-26,2020-06-11,323,2.00,2020,1
5,DIF,7000.0,103127.91,0.2610,1827.0,2019-07-24,2020-02-12,2020-03-03,223,2.90,2019,4
6,DIF,7000.0,103127.91,0.1656,1159.2,2019-07-24,2019-11-18,2019-12-04,133,3.08,2019,3
7,DIF,7000.0,103127.91,0.3469,2428.3,2019-07-24,2019-08-26,2019-09-11,49,17.54,2019,2


In [6]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div1 = df_grp
div1

name
DIF    14549.5
Name: div_amt, dtype: float64

In [7]:
file_name = 'div-' + name + '.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/div-DIF.csv',
 '\\Users\\User\\iCloudDrive\\div-DIF.csv',
 '\\Users\\User\\Dropbox\\div-DIF.csv')

In [8]:
df_merge[colt].to_csv(output_file)
df_merge[colt].to_csv(data_file)
df_merge[colt].to_csv(box_file)

### 2nd batch of purchase

In [9]:
div_year = 2019
div_quarter = 3
ids = '745'

In [10]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,DIF,5000.0,73662.8,2019-09-02


In [11]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name ='%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,235,DIF,2021,1,"100,000",0.2610,"26,100.00",2021-05-12,2021-06-04
1,197,DIF,2020,4,"80,000",0.2610,"20,880.00",2021-02-23,2021-03-12
2,190,DIF,2020,3,"70,000",0.2610,"18,270.00",2020-11-11,2020-12-04
3,3,DIF,2020,2,"60,000",0.2610,"15,660.00",2020-08-13,2020-09-08
4,6,DIF,2020,1,"52,000",0.2610,"13,572.00",2020-05-26,2020-06-11
5,31,DIF,2019,4,"52,000",0.2610,"13,572.00",2020-02-12,2020-03-03
6,36,DIF,2019,3,"52,000",0.1656,"8,611.20",2019-11-18,2019-12-04


In [12]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,DIF,5000.0,73662.8,0.2610,1305.0,2019-09-02,2021-05-12,2021-06-04,641,1.01,2021,1
1,DIF,5000.0,73662.8,0.2610,1305.0,2019-09-02,2021-02-23,2021-03-12,557,1.16,2020,4
2,DIF,5000.0,73662.8,0.2610,1305.0,2019-09-02,2020-11-11,2020-12-04,459,1.41,2020,3
3,DIF,5000.0,73662.8,0.2610,1305.0,2019-09-02,2020-08-13,2020-09-08,372,1.74,2020,2
4,DIF,5000.0,73662.8,0.2610,1305.0,2019-09-02,2020-05-26,2020-06-11,283,2.28,2020,1
5,DIF,5000.0,73662.8,0.2610,1305.0,2019-09-02,2020-02-12,2020-03-03,183,3.53,2019,4
6,DIF,5000.0,73662.8,0.1656,828.0,2019-09-02,2019-11-18,2019-12-04,93,4.41,2019,3


In [13]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div3 = df_grp
div3

name
DIF    8658.0
Name: div_amt, dtype: float64

In [14]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### 3rd batch of purchase

In [15]:
div_year = 2020
div_quarter = 2
ids = '781'

In [16]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net)-8000 AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,DIF,8000.0,109860.47,2020-08-01


In [17]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name ='%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,235,DIF,2021,1,"100,000",0.2610,"26,100.00",2021-05-12,2021-06-04
1,197,DIF,2020,4,"80,000",0.2610,"20,880.00",2021-02-23,2021-03-12
2,190,DIF,2020,3,"70,000",0.2610,"18,270.00",2020-11-11,2020-12-04
3,3,DIF,2020,2,"60,000",0.2610,"15,660.00",2020-08-13,2020-09-08


In [18]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,DIF,8000.0,109860.47,0.261,2088.0,2020-08-01,2021-05-12,2021-06-04,307,2.26,2021,1
1,DIF,8000.0,109860.47,0.261,2088.0,2020-08-01,2021-02-23,2021-03-12,223,3.11,2020,4
2,DIF,8000.0,109860.47,0.261,2088.0,2020-08-01,2020-11-11,2020-12-04,125,5.55,2020,3
3,DIF,8000.0,109860.47,0.261,2088.0,2020-08-01,2020-08-13,2020-09-08,38,18.26,2020,2


In [19]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div4 = df_grp
div4

name
DIF    8352.0
Name: div_amt, dtype: float64

In [20]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### 4th batch of purchase

In [21]:
div_year = 2020
div_quarter = 3
ids = '783'

In [22]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,DIF,10000.0,147325.59,2020-10-09


In [23]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name ='%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,235,DIF,2021,1,"100,000",0.2610,"26,100.00",2021-05-12,2021-06-04
1,197,DIF,2020,4,"80,000",0.2610,"20,880.00",2021-02-23,2021-03-12
2,190,DIF,2020,3,"70,000",0.2610,"18,270.00",2020-11-11,2020-12-04


In [24]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,DIF,10000.0,147325.59,0.261,2610.0,2020-10-09,2021-05-12,2021-06-04,238,2.72,2021,1
1,DIF,10000.0,147325.59,0.261,2610.0,2020-10-09,2021-02-23,2021-03-12,154,4.20,2020,4
2,DIF,10000.0,147325.59,0.261,2610.0,2020-10-09,2020-11-11,2020-12-04,56,11.55,2020,3


In [25]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div5 = df_grp
div5

name
DIF    7830.0
Name: div_amt, dtype: float64

In [26]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### 5th batch of purchase

In [27]:
div_year = 2020
div_quarter = 4
ids = '813'

In [28]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,DIF,10000.0,147325.59,2021-02-17


In [29]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name ='%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,235,DIF,2021,1,"100,000",0.2610,"26,100.00",2021-05-12,2021-06-04
1,197,DIF,2020,4,"80,000",0.2610,"20,880.00",2021-02-23,2021-03-12


In [30]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,DIF,10000.0,147325.59,0.261,2610.0,2021-02-17,2021-05-12,2021-06-04,107,6.04,2021,1
1,DIF,10000.0,147325.59,0.261,2610.0,2021-02-17,2021-02-23,2021-03-12,23,28.11,2020,4


In [31]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div6 = df_grp
div6

name
DIF    5220.0
Name: div_amt, dtype: float64

In [32]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### 6th batch of purchase

In [34]:
div_year = 2021
div_quarter = 1
ids = '824'

In [35]:
sql = '''
SELECT T.name, SUM(qty) AS ttl_qty, 
SUM(B.net) AS buy_amt, MIN(date) AS buy_date
FROM buys B
JOIN stocks T ON B.stock_id = T.id 
WHERE B.id IN (%s)
GROUP BY T.name'''
sql = sql % (ids)
buys = pd.read_sql(sql, conpf)
buys

,name,ttl_qty,buy_amt,buy_date
0,DIF,20000.0,294651.18,2021-03-02


In [36]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name ='%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,235,DIF,2021,1,"100,000",0.2610,"26,100.00",2021-05-12,2021-06-04


In [37]:
df_merge = pd.merge(buys, dividends, on='name', how='inner')
df_merge['div_amt'] = df_merge.ppu * df_merge.ttl_qty
df_merge['days'] = df_merge.p_date - df_merge.buy_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
df_merge['pct'] = round(df_merge['div_amt']/df_merge['buy_amt']*36500/df_merge['days'],2)
df_merge[cols]

,name,ttl_qty,buy_amt,ppu,div_amt,buy_date,x_date,p_date,days,pct,year,quarter
0,DIF,20000.0,294651.18,0.261,5220.0,2021-03-02,2021-05-12,2021-06-04,94,6.88,2021,1


In [38]:
df_grp =df_merge.groupby(['name'])['div_amt'].sum()
div7 = df_grp
div7

name
DIF    5220.0
Name: div_amt, dtype: float64

In [39]:
df_merge[colt].to_csv(output_file,mode='a',header=False)
df_merge[colt].to_csv(data_file,mode='a',header=False)
df_merge[colt].to_csv(box_file,mode='a',header=False)

### End of process

In [40]:
div_year = 2019
div_quarter = 2

In [43]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE name = '%s'
AND ((year = %s AND quarter >= %s) 
OR (year > %s))
ORDER BY name, year, quarter
"""
sql = sql % (name, div_year, div_quarter, div_year)
dividends = pd.read_sql(sql, conpf)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,43,DIF,2019,2,"47,000",0.3469,"16,304.30",2019-08-26,2019-09-11
1,36,DIF,2019,3,"52,000",0.1656,"8,611.20",2019-11-18,2019-12-04
2,31,DIF,2019,4,"52,000",0.2610,"13,572.00",2020-02-12,2020-03-03
3,6,DIF,2020,1,"52,000",0.2610,"13,572.00",2020-05-26,2020-06-11
4,3,DIF,2020,2,"60,000",0.2610,"15,660.00",2020-08-13,2020-09-08
5,190,DIF,2020,3,"70,000",0.2610,"18,270.00",2020-11-11,2020-12-04
6,197,DIF,2020,4,"80,000",0.2610,"20,880.00",2021-02-23,2021-03-12
7,235,DIF,2021,1,"100,000",0.2610,"26,100.00",2021-05-12,2021-06-04


In [44]:
dividends.div_amt.sum()

132969.5